In [8]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [9]:
def rmse(true,predicted):
    return np.sqrt(mean_squared_error(true,predicted))

In [10]:
train = pickle.load(open("../pickles/train_data.p",'rb'))
test = pickle.load(open("../pickles/test_data.p",'rb'))
target = pickle.load(open("../pickles/target.p",'rb'))
test_id = pd.read_csv("input/test.csv",header=0,usecols=["Id"])

In [11]:
print(train.shape)
print(test.shape)

(1456, 248)
(1459, 248)


In [12]:
x_train,x_val,y_train,y_val = train_test_split(train,target,random_state = 0)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1092, 248)
(364, 248)
(1092,)
(364,)


In [23]:
## Baggin Regressor 
bagging_regression = BaggingRegressor()
print(np.sqrt(abs(cross_val_score(bagging_regression,train,target,scoring='neg_mean_squared_error',cv=5).mean())))
bagging_regression = bagging_regression.fit(x_train,y_train)
predicted = bagging_regression.predict(x_val)
bagging_rmse = rmse(y_val,predicted)
print("Bagging Root Mean Square Error : ",bagging_rmse)

## Boosting Regressor
gradient_boosting = GradientBoostingRegressor(max_depth=5,learning_rate=0.09)
print(np.sqrt(abs(cross_val_score(gradient_boosting,train,target,scoring='neg_mean_squared_error',cv=5).mean())))
gradient_boosting = gradient_boosting.fit(x_train,y_train)
predicted_boosting = gradient_boosting.predict(x_val)
gradient_rmse = rmse(y_val,predicted_boosting)
print("Gradient Boosting Root Mean Square Error: ",gradient_rmse)

## Combined algorithm
combined = (predicted + predicted_boosting)/2
combined_rmse = rmse(y_val,combined)
print("Bagging and Boosting Combination Root Mean Square Error : ",combined_rmse)

## Prediction on Test Dataset
predicted_bagging = bagging_regression.predict(test)
predicted_boosting = gradient_boosting.predict(test)
combination = (predicted_bagging + predicted_boosting)/2

final = pd.DataFrame()
final["Id"] = test_id
final["SalePrice"] = combination
final.to_csv("../results/final_combined.csv",index=False)

27521.1138032
Bagging Root Mean Square Error :  26225.3588929
23843.6013581
Gradient Boosting Root Mean Square Error:  24399.2118892
Bagging and Boosting Combination Root Mean Square Error :  24162.1189024


27713.867167659046

25094.29674909293